# ECG Arrhythmia CNN+LSTM Demo
This notebook shows how to download data, build arrays, and train a small model.

In [ ]:
from pathlib import Path
import numpy as np
from ecgclf.data import download_mitbih, DEFAULT_DATA_DIR
from ecgclf.preprocess import build_arrays, CLASSES
from ecgclf.model import build_model, ensure_3d
print('DATA_DIR=', DEFAULT_DATA_DIR)

In [ ]:
# Optional: download (uncomment)
# download_mitbih(DEFAULT_DATA_DIR)

In [ ]:
X, y, rec_ids = build_arrays(DEFAULT_DATA_DIR)
X.shape, np.bincount(y, minlength=len(CLASSES))

In [ ]:
# Imports et chemins
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

from ecgclf.preprocess import build_arrays, CLASSES, stratified_kfold_indices, compute_class_weights
from ecgclf.model import build_model, ensure_3d

data_dir = Path.cwd() / 'data' / 'mitdb'
models_dir = Path.cwd() / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Charger le dataset (cela peut prendre plusieurs minutes)
X, y, rec_ids = build_arrays(data_dir)
Xc = ensure_3d(X)
print('X:', X.shape, 'Xc:', Xc.shape)
print('Classes:', CLASSES)
print('Distribution:', np.bincount(y, minlength=len(CLASSES)))

In [ ]:
# Visualisation de 5 segments (Normal, LBBB, RBBB, APB, PVC)
plt.figure(figsize=(12, 8))
shown = 0
for cls in range(len(CLASSES)):
    idxs = np.where(y == cls)[0]
    if idxs.size == 0:
        continue
    i = int(idxs[0])
    ax = plt.subplot(3, 2, shown + 1)
    ax.plot(X[i], lw=1.0)
    ax.set_title(f"Classe: {CLASSES[cls]} (idx={i})")
    ax.set_xlim([0, X.shape[1]])
    shown += 1
    if shown >= 5:
        break
plt.tight_layout()
plt.show()

In [ ]:
# Entraînement rapide sur 1 fold (quelques époques)
from tensorflow import keras
folds = list(stratified_kfold_indices(y, n_splits=2, random_state=0))
tr_idx, va_idx = folds[0]
x_tr, x_va = Xc[tr_idx], Xc[va_idx]
y_tr, y_va = y[tr_idx], y[va_idx]

class_weight = compute_class_weights(y_tr)
model = build_model(input_len=Xc.shape[1], n_classes=len(CLASSES), backend='keras')
# Assurer le LR = 1e-3
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_tr, y_tr, validation_data=(x_va, y_va), epochs=3, batch_size=64, class_weight=class_weight, verbose=2)

# Sauvegarder un modèle de démonstration
demo_model_path = models_dir / 'fold_1.keras'
model.save(demo_model_path)
print('Saved model to', demo_model_path)

In [ ]:
# Courbes d'entraînement: loss / accuracy
fig, axs = plt.subplots(1, 2, figsize=(12,4))
axs[0].plot(history.history['loss'], label='train')
axs[0].plot(history.history['val_loss'], label='val')
axs[0].set_title('Loss')
axs[0].legend()
axs[1].plot(history.history['accuracy'], label='train')
axs[1].plot(history.history['val_accuracy'], label='val')
axs[1].set_title('Accuracy')
axs[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
# Chargement du modèle sauvegardé et matrice de confusion
from tensorflow import keras
loaded = keras.models.load_model(demo_model_path)
probs = loaded.predict(x_va, verbose=0)
y_pred = np.argmax(probs, axis=1)
cm = confusion_matrix(y_va, y_pred, labels=list(range(len(CLASSES))))
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()